In [111]:
import qiskit
from qiskit.quantum_info import state_fidelity
import numpy as np
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy

I2 = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Random unitary generator
def random_unitary(n):
    A = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    Q, _ = np.linalg.qr(A)
    return Q

L = 4
# construct Hamiltonian
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.IsingHamiltonian(field, 1, 0, 0).as_matrix().toarray()
#hamil = qib.HeisenbergHamiltonian(field, (1,1,1), (0,0,0)).as_matrix().toarray()
perms = [[i for i in range(L)], [i for i in range(1, L)]+[0]]
#perms = [[0, 1, 2, 3]]
U = scipy.linalg.expm(-1j*hamil)
U_back = scipy.linalg.expm(1j*hamil)
cU = U_back

In [114]:
import sys
sys.path.append("../2qubit-ccU")
from utils import otimes, applyG
from ansatz import ansatz, ansatz_grad_vector
from hessian import ansatz_hessian_matrix
from optimize import err

V1 = np.kron(X, I2)
V2 = np.kron(I2, I2)
W1 = V1
W2 = V2
Glist_opt = [V1, V2, W1, W2]
#Glist_opt = [V1, W1]

print(np.linalg.norm(cU - ansatz(Glist_opt, U, L, perms), ord=2))
print(err(Glist_opt, U, L, perms, cU))

0.0
-16.0


In [115]:
hess_opt = -ansatz_hessian_matrix(Glist_opt, cU, U, L, perms, unprojected=False)
np.all(np.linalg.eigvals(hess_opt) >= -1e-3)

True

In [116]:
# d/G_j d/dG_i f(G)
from utils import polar_decomp

def numerical_hessian(Glist, cU, U, L, perms, i, j, epsilon=1e-6):
    """Numerically compute d/dW1 of projected gradient dL/dV1 (Riemannian)."""
    numerical_H = []

    for _ in range(16):
        # Create unit vector in 16D real tangent space
        Z_real = np.zeros(16)
        Z_real[_] = 1.0
        Z = real_to_antisymm(Z_real.reshape(4, 4))  # 4x4 anti-Hermitian direction

        # Perturb G_j on the manifold
        #Gj_plus  = Glist[j] + epsilon * Glist[j] @ Z
        #Gj_minus = Glist[j] - epsilon * Glist[j] @ Z
        Gj_plus  = Glist[j] @ scipy.linalg.expm(+epsilon*Z)
        Gj_minus = Glist[j] @ scipy.linalg.expm(-epsilon*Z)

    
        if i==j:
            grad_plus  = ansatz_grad_vector(Glist[:j]+[Gj_plus]+Glist[j+1:], cU, U, L, perms, unprojected=True, flatten=False)[i]
            grad_minus = ansatz_grad_vector(Glist[:j]+[Gj_minus]+Glist[j+1:], cU, U, L, perms, unprojected=True, flatten=False)[i]
            dgrad = (grad_plus - grad_minus) / (2 * epsilon)  # shape (16,)
            G = dgrad.reshape(4, 4)

            # TODO: Is this projection correct?
            V = Glist[j]
            Z = V @ Z
            G = project_unitary_tangent(V, G)
            grad = ansatz_grad_vector(Glist, cU, U, L, perms, flatten=False, unprojected=True)[i]
            G -= 0.5 * (Z @ grad.conj().T @ V + V @ grad.conj().T @ Z)
            if not np.allclose(Z, project_unitary_tangent(V, Z)):
                G -= 0.5 * (Z @ V.conj().T + V @ Z.conj().T) @ grad
            G = antisymm_to_real(antisymm( V.conj().T @ G ))
        else:
            grad_plus  = ansatz_grad_vector(Glist[:j]+[Gj_plus]+Glist[j+1:], cU, U, L, perms, unprojected=False, flatten=False)[i]
            grad_minus = ansatz_grad_vector(Glist[:j]+[Gj_minus]+Glist[j+1:], cU, U, L, perms, unprojected=False, flatten=False)[i]
            dgrad = (grad_plus - grad_minus) / (2 * epsilon)  # shape (16,)
            G = dgrad.reshape(4, 4)
            
        numerical_H.append(G)
    
    return np.array(numerical_H)  # shape: (16, 4, 4)

In [125]:
from utils import real_to_antisymm, antisymm_to_real, antisymm

i, j = 2, 2 # 0, 1, 2, 3
Glist = [random_unitary(4) for _ in range(2 * len(perms))]
H = ansatz_hessian_matrix(Glist, cU, U, L, perms, unprojected=False, flatten=False)
grad = []
for _ in range(16):
    grad.append(H[i, :, j, _].reshape(4,4))
analytical = np.array(grad)

numerical = numerical_hessian(Glist, cU, U, L, perms, i, j)

#print("Numerical 4x4 matrix:\n", numerical[0])
#print("Analytical 4x4 matrix:\n", analytical[0])
print("Difference norm:", np.linalg.norm(numerical - analytical))

Difference norm: 1.0313535054521224e-09
